# Gemini & ChatGPT

In [36]:
print(
"""
***************************************
00_Initialize_LLM_Jerry.ipynb
2024/11/09 005 version
***************************************
"""
)


***************************************
00_Initialize_LLM_Jerry.ipynb
2024/10/27 005 version
***************************************



In [ ]:
# %env GOOGLE_LOGIN=TRUE
# %env REINSTALL=TRUE

In [6]:
%%shell
echo "REINSTALL: $REINSTALL"
if [ "$REINSTALL" == "TRUE" ]
then
  pip install \
  langchain==0.3.4 \
  langchain-community==0.3.3 \
  langchain_openai==0.2.1 \
  langchain-google-genai==2.0.1 \
  langchain-anthropic==0.2.3 \
  gradio==5.4.0 \
  google-cloud-aiplatform==1.70.0 \
  langchain-google-vertexai==2.0.3 \
  langgraph==0.2.35 langsmith==0.1.132 \
  langchain_google_community==2.0.1 \
  google-cloud-discoveryengine==0.13.1 \
  llama-index==0.11.19 \
  wikipedia==1.4.0 \
  google-search-results==2.4.2 \
  youtube-search==2.1.2 \
  chromadb==0.5.15 \
  langchain-chroma==0.1.4 \
  datasets==3.1.0 \
  pinecone==5.3.1 \
  langchain_pinecone==0.2.0 \
  sentence-transformers==3.2.1 \
  
fi

REINSTALL: TRUE
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=d995c1140e1101988ef7d658a603f5b645d591942242e4a8029d6c1d83d67004
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [ ]:
import os

print(f"GOOGLE_LOGIN: {os.environ.get('GOOGLE_LOGIN')}")
if "GOOGLE_LOGIN" in os.environ and os.environ["GOOGLE_LOGIN"] == "TRUE":
  from google.colab import auth
  auth.authenticate_user()

In [ ]:
!pip list | grep langchain

In [ ]:
from google.colab import userdata
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
import json

# public colab인지 일반 jupyter server인지 확인
colab = True
credentials = {}
try:
    api_key = userdata.get('openai-key')
except:
    colab = False

print("Colab:", colab)

if colab == False:
    print("Colab에 Secret이 없으므로, lang-chain/credentials.json 파일을 사용합니다.")
    print(f"credentials.json 예제: https://github.com/javalove93/langchain_study/raw/refs/heads/main/credentials-sample.json")
    with open("lang-chain/credentials.json") as f:
        credentials = json.load(f)

# Open AI (불필요 하면 제거)
if colab:
    api_key = userdata.get('openai-key')
else:
    api_key = credentials.get('openai-key')

os.environ['OPENAI_API_KEY'] = api_key
API_KEY = api_key

openai_llm = ChatOpenAI(openai_api_key=API_KEY)

# Anthropic (불필요 하면 제거)
if colab:
    anthropic_api_key = userdata.get('anthropic-key')
else:
    anthropic_api_key = credentials.get('anthropic-key')

os.environ["ANTHROPIC_API_KEY"] = anthropic_api_key

anthropic_llm = ChatAnthropic(model="claude-3-opus-20240229")

# Google Gemini (불필요 하면 제거)
if colab:
    google_api_key = userdata.get('gemini-api-key')
else:
    google_api_key = credentials.get('gemini-api-key')

os.environ["GOOGLE_API_KEY"] = google_api_key

gemini_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
# setup the gemini pro
# https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versioning?hl=ko#stable-versions-available.md
# text-embedding-004	2024년 5월 14일	해당 없음
# text-multilingual-embedding-002	2024년 5월 14일	해당 없음
# https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-embeddings-api?hl=ko
# 멀티모달 임베딩	multimodalembedding@001
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="text-multilingual-embedding-002")

llm = gemini_llm

# Google Search Wrapper (불필요 하면 제거)
if colab:
    SERPER_API_KEY = userdata.get('google-serper-key')
else:
    SERPER_API_KEY = credentials.get('google-serper-key')

os.environ["SERPER_API_KEY"] = SERPER_API_KEY

def print_message(aiMessage, messageOnly=True):
    print("RESPONSE: {}".format(aiMessage.content))
    if messageOnly == False:
        print(json.dumps(aiMessage.additional_kwargs, indent=2))
        print(json.dumps(aiMessage.response_metadata, indent=2))
        print(json.dumps(aiMessage.usage_metadata, indent=2))

# Lang Smith Tracing (API Key 발급받아야 하며, 불필요하면 제거)
%env LANGCHAIN_TRACING_V2=true
%env LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
if colab:
    os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith')
else:
    os.environ["LANGCHAIN_API_KEY"] = credentials.get('langsmith')
%env LANGCHAIN_PROJECT=langgraph-test

# Google Programmable Search API
if colab:
    os.environ["GOOGLE_API_KEY"] = userdata.get('google-search-api-key')
    os.environ["GOOGLE_CSE_ID"] = userdata.get('google-search-cse')
else:
    os.environ["GOOGLE_API_KEY"] = credentials.get('google-search-api-key')
    os.environ["GOOGLE_CSE_ID"] = credentials.get('google-search-cse')

# PINECONE API
if colab:
    os.environ["PINECONE_API_KEY"] = userdata.get('pinecone.io')
else:
    os.environ["PINECONE_API_KEY"] = credentials.get('pinecone.io')

# https://smith.langchain.com